In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.pipeline import Pipeline as imb_pipeline
from imblearn.under_sampling import RandomUnderSampler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

data = pd.read_csv("healthcare-dataset-stroke-data.csv")

balanced_data = data.dropna()
balanced_data = balanced_data.drop(columns= "id")
balanced_data = balanced_data[balanced_data.age > 20]
balanced_data = balanced_data.drop(balanced_data.index[balanced_data["gender"] == "Other"])
balanced_data = balanced_data.drop(balanced_data.index[balanced_data["work_type"] == "Never_worked"])

features = ['gender', 'age', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'avg_glucose_level', "bmi", "smoking_status"]
cat_columns = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status', 'hypertension', 'heart_disease']
num_columns = ['age', 'avg_glucose_level', 'bmi']

bal_data_train, bal_data_test = train_test_split(balanced_data, train_size = 0.8, random_state=1, stratify= balanced_data["stroke"])
bal_data_train, bal_data_val = train_test_split(bal_data_train, train_size = 0.8, random_state=1)

X_train = bal_data_train.drop('stroke', axis = 1)
y_train = bal_data_train['stroke']

X_val = bal_data_val.drop('stroke', axis = 1)
y_val = bal_data_val['stroke']

X_test = bal_data_test.drop('stroke', axis = 1)
y_test = bal_data_test['stroke']

In [2]:
preprocessing = ColumnTransformer([
    ('one-hot-encoder', OneHotEncoder(sparse_output=False), cat_columns),
    ('scaler', StandardScaler(), num_columns)
])

In [9]:
X_train2 = preprocessing.fit_transform(X_train)
X_val2 = preprocessing.transform(X_val)
X_test2 = preprocessing.transform(X_test)


(2500, 20)

Initial model starts at 0.07 recall

In [8]:
import tensorflow as tf

tf.random.set_seed(1)
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=20))
model.add(tf.keras.layers.Dense(512, 'relu'))
model.add(tf.keras.layers.Dense(512, 'relu'))
model.add(tf.keras.layers.Dense(1, "sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['Recall'])

#model.summary()
model_history = model.fit(X_train2, y_train, validation_data=(X_val2, y_val), epochs=100)

Epoch 1/100
79/79 [==============================] - 1s 5ms/step - loss: 0.1970 - recall: 0.0000e+00 - val_loss: 0.2016 - val_recall: 0.0000e+00
Epoch 2/100
79/79 [==============================] - 0s 3ms/step - loss: 0.1697 - recall: 0.0000e+00 - val_loss: 0.1980 - val_recall: 0.0000e+00
Epoch 3/100
79/79 [==============================] - 0s 3ms/step - loss: 0.1661 - recall: 0.0000e+00 - val_loss: 0.1973 - val_recall: 0.0263
Epoch 4/100
79/79 [==============================] - 0s 3ms/step - loss: 0.1597 - recall: 0.0000e+00 - val_loss: 0.1992 - val_recall: 0.0000e+00
Epoch 5/100
79/79 [==============================] - 0s 3ms/step - loss: 0.1569 - recall: 0.0000e+00 - val_loss: 0.2057 - val_recall: 0.0263
Epoch 6/100
79/79 [==============================] - 0s 3ms/step - loss: 0.1512 - recall: 0.0312 - val_loss: 0.2146 - val_recall: 0.0000e+00
Epoch 7/100
79/79 [==============================] - 0s 3ms/step - loss: 0.1475 - recall: 0.0547 - val_loss: 0.2251 - val_recall: 0.0789
Epoch

In [10]:
model.evaluate(X_test2, y_test)

25/25 [==============================] - 0s 1ms/step - loss: 1.1522 - recall: 0.0952


[1.1521899700164795, 0.095238097012043]

https://imbalanced-learn.org/stable/miscellaneous.html

https://www.freecodecamp.org/news/binary-classification-made-simple-with-tensorflow/
